<a href="https://colab.research.google.com/github/cps41/health-informatics-portfolio/blob/main/bigquery_data_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio Assignment- Data Modeling and Cleaning

## Background
The goal of this project is to practice and demonstrate the following:
- Manage large-scale clinical data sets using cloud-based tools such as a data warehouse and notebooks.
- Apply descriptive statistical methods to analyze health data with summary statistics, visualizations, and pattern/trend discovery.
- Build entity-relationship diagrams, apply primary and foreign keys, and understand cardinality to evaluate the relationships between different data sets.

## The Question
We will be using our skills to design, populate, and validate a dimensional model. The intention is to use this model to answer an analytical question. This question was answered in a [previous project](https://colab.research.google.com/drive/1KRavUnNEwF3j2u2dGINnp466zC_3Qc36?usp=sharing) using a different dataset and approach.

**How many prescriptions were for metformin each month?**

## The Model
We will begin by creating a dimensional data model using the Kimball data modeling techniques. This means we will need fact and dimension tables. Our model will be based on FHIR data.
- Fact Tables
    - __Encounter__
    - __Medication Administration__
- Dimension Tables
    - __Patient__
    - __Medication__
    - __Encounter Type__

### FHIR Resources
The HL7 FHIR standard uses entities know as _resources_. Our model will be based on the following resources (HL7 2023):
- Encounter
  - An encounter represents an event that ocurred to a patient regarding a healthcare service. It contains information about the encounter such as setting,

### Columns
To choose what columns we want, we will need to work backwards a bit. While we could include all of the elements of these FHIR resources, we want to optimize our solution for the question we are answering.

### Entity Relationship Diagram
Let's demonstrate this model using an ERD, or entity relationship diagram. This will be generated using mermaid.js and Python based on the Mermaid documentation (Mermaid n.d.).

You can clearly see we've limited our details down and are following the principles of the Kimball method to have a one to many relationship between fact and dimension tables, as well as having surrofate ids in the dimension tables (Kimball 2013).

In [ ]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

mm("""
erDiagram
	PATIENT {
		int id
		string patientId
    string firstName
    string lastName
	}
	MEDICATION {
		int id
		int medicationId
		string medicationName
	}
	ENCOUNTER_TYPE {
		int encounterTypeId
		string encounterTypeCode
		string encounterTypeName
	}
	ENCOUNTER {
		string encounterId
		int encounterTypeId
		int patientId
		string status
	}
	MEDICATION_ADMINISTRATION {
		string medicationAdministrationId
		int patientId
		int medicationId
		string encounterId
		int month
		int year
		string status
	}

	ENCOUNTER }|--|| ENCOUNTER_TYPE : is_type
	MEDICATION_ADMINISTRATION }|--|{ ENCOUNTER : relates_to
	MEDICATION_ADMINISTRATION }|--|| PATIENT : administers_to
	ENCOUNTER }|--|| PATIENT : happens_to
	MEDICATION_ADMINISTRATION }|--|| MEDICATION : administers
""")

## Table Creation
Let's use the Synthea dataset to populate our tables. First, we need to set up our client connection to BigQuery.

In [ ]:
from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

project_id = "assignment-1-448801"
client = bigquery.Client(project=project_id);

Authenticated


Before we can create tables, we need to create a dataset using the BigQuery library to put them in.

In [ ]:
dataset_id = "portfolio_data_modeling"
dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)
print(f"Created dataset {dataset.full_dataset_id}")

Created dataset assignment-1-448801:portfolio_data_modeling


### Dimension Tables
The dimension tables are used to provide further context to data in the fact tables (Kimball 2013, 13-15). Let's begin with the Encounter Type dimension. This adds further information about encounter types to the Encounter table and is linked via the `encounterTypeId`.

I will use a SQL query as I find it more readable than the API calls, though I'd likely use those instead if this were infrastructure as code.

#### Encounter Type Table

In [ ]:
# First create the table if it does not already exist
# Follow the schema presented in our ERD
client.query(
f'''  CREATE OR REPLACE TABLE {dataset.dataset_id}.encounter_type (
    encounterTypeId INT,
    encounterTypeCode STRING,
    encounterTypeName STRING
  );
''').result()

encounter_type_table = client.get_table(f"{dataset.dataset_id}.encounter_type")
encounter_type_table_name = f"{dataset.dataset_id}.{encounter_type_table.table_id}"
print(f"Successfully created table {encounter_type_table}")

NameError: name 'client' is not defined

Now that we've created our table, we should perform some table optimization. While typical indexing is not included in BigQuery, search indexes are. These take advantage of BigQueries optimization functionality to improve the efficiency of queries that use the SEARCH function on a column. The SEARCH function is for semi or unstructured text matching (Google Cloud 2025). Since we may want to query certain types of encounters, it makes sense for us to put one on the encounterTypeName. We will only be doing exact matches on the encounterTypeId, so it is unneccessary to place one on that column.

In [ ]:
# Add optimization features
client.query(
f'''
  -- Make searching for text in the encounterTypeName more efficient
  CREATE SEARCH INDEX encounter_type_index
  ON `{encounter_type_table_name}`(encounterTypeName);
''')

QueryJob<project=assignment-1-448801, location=US, id=6bcbc378-48b0-477a-938a-2ddb1aed7fb6>

The fhir_synthea dataset does not have an Encounter Type table, but it does have an encounter table that references encounter types in it's nested record values. We will populate our table by extracting the desired values for *encounterTypeCode* and *encounterTypeName* to align with our ERD.

The type's code is a unique identifier for an encounter type, thus will be used as our *encounterTypeCode*. The type's display is the human readable display name and will be used as our *encounterTypeName*.

##### *Assumption*
The FHIR Encounter resource inherently goes against the one to many rule in the Kimball dimension table method, by having multiple encounter types per encounter. The alternative approach would be to coalesce these into a concatenated string or array, or have multiple rows per encounterTypeId and use a bridge table. For this project we will just be taking the first type value as it feels more appropriate without further requirements for the table's purpose.

In [ ]:
# Populate the table by extracting encounter type info from the nested values
# in the fhir_synthea.encounter table
client.query(
f'''
  INSERT INTO {encounter_type_table_name} (encounterTypeId, encounterTypeCode, encounterTypeName)
  (
    SELECT
      -- Create unique incremental id for rows as a surrogate id
      ROW_NUMBER() OVER (ORDER BY encounterTypeCode, encounterTypeName) as encounterTypeId,
      encounterTypeCode,
      encounterTypeName
    FROM (
      SELECT DISTINCT
        -- Flatten encounter.type.coding.code into encounterTypeCode
        `type`[SAFE_OFFSET(0)].`coding`[SAFE_OFFSET(0)].code as encounterTypeCode,
        -- Flatten encounter.type.coding.display into encounterTypeName
        `type`[SAFE_OFFSET(0)].`coding`[SAFE_OFFSET(0)].display as encounterTypeName
      FROM `bigquery-public-data.fhir_synthea.encounter`
      -- Eliminate missing values
      WHERE `type`[SAFE_OFFSET(0)].`coding`[SAFE_OFFSET(0)].display IS NOT NULL
        AND `type`[SAFE_OFFSET(0)].`coding`[SAFE_OFFSET(0)].code IS NOT NULL
    )
  )
''').result()

Let's do a quick check to see some of the data we just handled.

In [ ]:
# Quick check to see if rows populated
client.query(
f'''
  SELECT *
  FROM {encounter_type_table_name}
  LIMIT 10
''').to_dataframe()

,encounterTypeId,encounterTypeCode,encounterTypeName
0,5,183452005,Encounter Inpatient
1,15,185347001,Encounter for problem (procedure)
2,12,185345009,Encounter for symptom (procedure)
3,30,371883000,Outpatient procedure (procedure)
4,21,270427003,Patient-initiated encounter
5,17,185349003,Encounter for check up
6,41,67799006,Diagnosis of cystic fibrosis using sweat test ...
7,26,308646001,Death Certification
8,31,390906007,Follow-up encounter
9,42,698314001,Consultation for treatment


##### Validation
Now let's perform some validation to see if the data matches our expectations.

In [ ]:
client.query(
f'''
  -- Get total number of rows
  SELECT COUNT(*) as totalRows,
    -- Get total unique ids
    COUNT(DISTINCT encounterTypeId) as totalIds,
    -- Get total unique codes
    COUNT(DISTINCT encounterTypeId) as totalCodes,
    -- Get total unique names
    COUNT(DISTINCT encounterTypeName) as totalNames
  FROM {encounter_type_table_name}
''').to_dataframe()

,totalRows,totalIds,totalCodes,totalNames
0,43,43,43,43


As expected, we get the same count across all, meaning all values are unique without duplicates. Now lets check for NULL values.

In [ ]:
client.query(
    f'''
      SELECT COUNT(*) as totalNulls
      FROM {encounter_type_table_name}
      WHERE encounterTypeId IS NULL
        OR encounterTypeCode IS NULL
        OR encounterTypeName IS NULL
    '''
).to_dataframe()

,totalNulls
0,0


We can now see there are no NULL values for any of our required elements.

#### Patient Table
Let's move on to our Patient table.

In [ ]:
# First create the table if it does not already exist
# Follow the schema presented in our ERD
client.query(
f'''  CREATE OR REPLACE TABLE {dataset.dataset_id}.patient (
    id INT,
    patientId STRING,
    firstName STRING,
    lastName STRING
  );
''')

patient_table = client.get_table(f"{dataset.dataset_id}.patient")
patient_table_name = f"{dataset.dataset_id}.{patient_table.table_id}"
print(f"Successfully created table {patient_table}")

Successfully created table assignment-1-448801.portfolio_data_modeling.patient


Unlike the Encounter Types table, we most likely won't be searching any of the Patient table columns, so there is no reason to add a search index. There are other forms of table optimization, but this table will largely just be additive for details, so it is not worth extra overhead.

The fhir_synthea dataset has a Patient table which makes this a little more straightforward than the Encounter Type table. We can directly match the columns as (HL7 2023):

- *patientId* = id
- *firstName* = name.given
- *lastName* = name.family

##### *Assumption*
I am making the same assumption as the encounter type table regarding multiple names per patientId. I am also making the assumption that it is possible a patient's name is unknown, but we still want to be able to link to them, thus the *firstName* and *lastName* are nullable. It is unusual to search by name over id.

In [ ]:
# Populate the table by extracting patient info from the nested values
# in the fhir_synthea.patient table
client.query(
f'''
  INSERT INTO {patient_table_name} (id, patientId, firstName, lastName)
  (
    SELECT
      -- Create unique incremental id for rows as a surrogate id
      ROW_NUMBER() OVER (ORDER BY patientId) as id,
      patientId,
      firstName,
      lastName
    FROM
    (
      SELECT
        DISTINCT id as patientId,
        -- Unwrap names from nested values
        `name`[SAFE_OFFSET(0)].`given`[SAFE_OFFSET(0)] as firstName,
        `name`[SAFE_OFFSET(0)].family as lastName
      FROM `bigquery-public-data.fhir_synthea.patient`
      -- Eliminate missing values, first and last name may be null
      WHERE id IS NOT NULL
    )
  )
''').result()

Let's do a quick check to see some of the data we just handled.

In [ ]:
# Quick check to see if rows populated
client.query(
f'''
  SELECT *
  FROM {patient_table_name}
  LIMIT 10
''').to_dataframe()

,id,patientId,firstName,lastName
0,36,0001f6ae-97a3-4d71-8ba6-6d975d2318d6,Gayla444,Quitzon246
1,472,001b11b6-0a85-4b06-97b9-058c62645ac9,Malka46,Nolan344
2,514,001d5027-e23c-4757-91de-905a9c94efcc,Marlin805,Rolfson709
3,649,002521e3-dcb2-4bbb-8137-3b956a4d0c9a,Jovita355,Keebler762
4,786,002d50b2-5f85-4b45-8b20-7f1d28e14faf,Frederica279,Lindgren255
5,922,0033b630-ea71-4976-9f28-ce30ab912b46,Mozella148,Schmitt836
6,1234,0044dcba-1ab2-4bc0-9523-4d17537e6ee0,Alissa315,McGlynn426
7,1276,00470425-9b4e-4556-a56b-32e1a7cd484d,Enoch803,Cummerata161
8,1315,0048b12f-5ce6-400d-97eb-12aefb6e8ad8,Sunni15,Goyette777
9,1423,004ebcbd-f0bb-48cf-8827-5078883eccc6,Jacobo456,Rojas511


##### Validation
Now let's perform some validation to see if the data matches our expectations.

In [ ]:
client.query(
f'''
  -- Get total number of rows
  SELECT COUNT(*) as totalRows,
    -- Get totals for unique columns
    COUNT(DISTINCT id) as totalIds,
    COUNT(DISTINCT patientId) as totalPatientIds
  FROM {patient_table_name}
''').to_dataframe()

,totalRows,totalIds,totalPatientIds
0,1176837,1176837,1176837


The totals for distinct firstNames, and lastNames may have duplicates, so we do not care to check the count for them as it will most likely be less than the total rows. Let's confirm no NULL values.

In [ ]:
client.query(
    f'''
      SELECT COUNT(*) as totalNulls
      FROM {patient_table_name}
      WHERE id IS NULL
      OR patientId IS NULL
    '''
).to_dataframe()

,totalNulls
0,0


#### Medication Table
Lastly for our dimension tables is the Medication table.

In [ ]:
# First create the table if it does not already exist
# Follow the schema presented in our ERD
client.query(
f'''  CREATE OR REPLACE TABLE {dataset.dataset_id}.medication (
    medicationId INT,
    medicationCode STRING,
    medicationName STRING
  );
''')

medication_table_name = f"{dataset.dataset_id}.medication"
medication_table = client.get_table(medication_table_name)
print(f"Successfully created table {medication_table}")

Successfully created table assignment-1-448801.portfolio_data_modeling.medication


Since we for sure want to query certain types of medication, particularly to answer our question about metformin, it makes sense for us to put a search index on the *medicationName*. We will only be doing exact matches on the *medicationId*, so it is unneccessary to place one on that column.

In [ ]:
# Add optimization features
client.query(
f'''
  -- Make searching for text in the medicationName more efficient
  CREATE SEARCH INDEX medication_index
  ON `{medication_table_name}`(medicationName);
''')

QueryJob<project=assignment-1-448801, location=US, id=4684bbdb-d24a-4209-993e-f759eaf9b716>

The fhir_synthea dataset does not have a Medication table, but it does have a Medication Request table that references medications in it's nested record values. Similar to the Encounter Types, we will populate our table by extracting the desired values for *medicationId* and *medicationName* to align wiht our ERD.

##### *Assumption*
I am making the same assumption as the encounter type and patient table regarding handling multiple names per medication.

The type's code is an identifier for a medication, thus will be used as our *medicationCode*. The medication's code display is the human readable display name and will be used as our *medicationName*.

In [ ]:
# Populate the table by extracting medication info from the nested values
# in the fhir_synthea.medication table
client.query(
f'''
  INSERT INTO {medication_table_name} (medicationId, medicationCode, medicationName)
  (
    SELECT
      -- Create unique incremental id for rows as a surrogate id
      ROW_NUMBER() OVER (ORDER BY medicationCode, medicationName) as medicationId,
      medicationCode,
      medicationName
    FROM
    (
      SELECT DISTINCT
        -- Flatten medication.codeableConcept.coding.code into medicationCode
        medication.codeableConcept.`coding`[SAFE_OFFSET(0)].code as medicationCode,
        -- Flatten medication.codeableConcept.coding.display into medicationName
       medication.codeableConcept.`coding`[SAFE_OFFSET(0)].display as medicationName
      FROM `bigquery-public-data.fhir_synthea.medication_request`
      -- Eliminate missing values
      WHERE medication.codeableConcept.`coding`[SAFE_OFFSET(0)].code IS NOT NULL
        AND medication.codeableConcept.`coding`[SAFE_OFFSET(0)].display IS NOT NULL
    )
  );
''').result()

Let's do a quick check to see some of the data we just handled.

In [ ]:
# Quick check to see if rows populated
client.query(
f'''
  SELECT *
  FROM {medication_table_name}
  LIMIT 10
''').to_dataframe()

,medicationId,medicationCode,medicationName
0,21,1359133,Estrostep Fe 28 Day Pack
1,64,259255,Atorvastatin 80 MG Oral Tablet
2,89,316052,Hydrochlorothiazide 6.25 MG
3,107,705129,Nitroglycerin 0.4 MG/ACTUAT Mucosal Spray
4,47,1870230,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...
5,58,205532,Pulmozyme (Dornase Alfa)
6,6,1049221,Acetaminophen 325 MG / Oxycodone Hydrochloride...
7,14,1091392,Methylphenidate Hydrochloride 20 MG Oral Tablet
8,37,1734340,Etoposide 100 MG Injection
9,106,665078,Loratadine 5 MG Chewable Tablet


##### Validation
Now let's perform some validation to see if the data matches our expectations.

In [ ]:
client.query(
f'''
  -- Get total number of rows
  SELECT COUNT(*) as totalRows,
    -- Get total unique ids
    COUNT(DISTINCT medicationId) as totalIds,
    -- Get total unique codes
    COUNT(DISTINCT medicationCode) as totalCodes,
    -- Get total unique names
    COUNT(DISTINCT medicationName) as totalNames
  FROM {medication_table_name}
''').to_dataframe()

,totalRows,totalIds,totalCodes,totalNames
0,150,150,148,150


And here we can see there must be an instance where a code appears with multiple names that we must correct and cleanup. This is why we do validation! Here is our new query with an addition to remove duplicates.

In [ ]:
# Clear out data
client.query(
  f'''
    DELETE FROM {medication_table_name}
    WHERE medicationId IS NOT NULL
  '''
).result()

# Populate the table by extracting medication info from the nested values
# in the fhir_synthea.medication table
client.query(
f'''
  INSERT INTO {medication_table_name} (medicationId, medicationCode, medicationName)
  (
    SELECT
      -- Create unique incremental id for rows as a surrogate id
      ROW_NUMBER() OVER (ORDER BY medicationCode, medicationName) as medicationId,
      medicationCode,
      medicationName
    FROM
    (
      SELECT DISTINCT
        -- Numerate rows based on a partition over the medicationCode so we can later eliminate duplicates
        ROW_NUMBER() OVER
          (PARTITION BY medication.codeableConcept.`coding`[SAFE_OFFSET(0)].code
          ORDER BY medication.codeableConcept.`coding`[SAFE_OFFSET(0)].display) as rowNum,
        -- Flatten medication.codeableConcept.coding.code into medicationCode
        medication.codeableConcept.`coding`[SAFE_OFFSET(0)].code as medicationCode,
        -- Flatten medication.codeableConcept.coding.display into medicationName
       medication.codeableConcept.`coding`[SAFE_OFFSET(0)].display as medicationName
      FROM `bigquery-public-data.fhir_synthea.medication_request`
      -- Eliminate missing values
      WHERE medication.codeableConcept.`coding`[SAFE_OFFSET(0)].code IS NOT NULL
        AND medication.codeableConcept.`coding`[SAFE_OFFSET(0)].display IS NOT NULL
    )
    -- Keep only the first name for each code
    WHERE rowNum = 1
  );
''').result()

In [ ]:
client.query(
f'''
  -- Get total number of rows
  SELECT COUNT(*) as totalRows,
    -- Get total unique ids
    COUNT(DISTINCT medicationId) as totalIds,
    -- Get total unique codes
    COUNT(DISTINCT medicationCode) as totalCodes,
    -- Get total unique names
    COUNT(DISTINCT medicationName) as totalNames
  FROM {medication_table_name}
''').to_dataframe()

,totalRows,totalIds,totalCodes,totalNames
0,148,148,148,148


And now we have validated that our table is deduplicated! Lastly, let's check for duplicates.

In [ ]:
client.query(
    f'''
      SELECT COUNT(*) as totalNulls
      FROM {medication_table_name}
      WHERE medicationId IS NULL
        OR medicationCode IS NULL
        OR medicationName IS NULL
    '''
).to_dataframe()

,totalNulls
0,0


### Fact Tables
The fact tables are what hold all of the records that relate to real world instances in a 1:1 relationship. (Kimball 2013, 10-12). Let's begin with the Encounter fact table. To maintain granularity (Kimball 2013, 11), we will limit the supporting columns to identifiers that link to dimension tables.

#### Encounter Table
The Encounter fact table will contain records that link back to instances of the Encounter FHIR resource which in our case, will be found in the fhir_synthea.encounter table.  

In [ ]:
# First create the table if it does not already exist
# Follow the schema presented in our ERD
client.query(
f'''  CREATE OR REPLACE TABLE {dataset.dataset_id}.encounter (
    encounterId STRING,
    encounterTypeId INT,
    patientId INT,
    status STRING
  );
''')

encounter_table = client.get_table(f"{dataset.dataset_id}.encounter")
encounter_table_name = f"{dataset.dataset_id}.{encounter_table.table_id}"
print(f"Successfully created table {encounter_table}")

Successfully created table assignment-1-448801.portfolio_data_modeling.encounter


We will just be joining dimension tables to the ids, so we don't have to add a search index here.

In [ ]:
# Populate the table by extracting the surrogate ids from the dimension tables
client.query(
f'''
  INSERT INTO {encounter_table_name} (encounterId, encounterTypeId, patientId, status)
  (
    SELECT
      DISTINCT
      encounter.id as encounterId,
      encounter_type.encounterTypeId,
      patient.id,
      encounter.status
    FROM `bigquery-public-data.fhir_synthea.encounter` as encounter
      -- Match encounterTypeCodes to retrieve encounterTypeId
      JOIN `portfolio_data_modeling.encounter_type` as encounter_type
      ON encounter_type.encounterTypeCode = encounter.`type`[SAFE_OFFSET(0)].`coding`[SAFE_OFFSET(0)].code
      -- Match patientId to retrieve id
      JOIN `portfolio_data_modeling.patient` as patient
      ON patient.patientId = encounter.subject.patientId
  );
''').result()

#### Medication Administration Table
The Medication Administration fact table will contain records that link back to instances of the Medication Request FHIR resources that were completed, which in our case, will be found in the fhir_synthea.medication_request table.

In [ ]:
# First create the table if it does not already exist or replace it
# Follow the schema presented in our ERD
client.query(
f'''  CREATE OR REPLACE TABLE {dataset.dataset_id}.medication_administration (
    medicationAdministrationId STRING,
    patientId INT,
    medicationId INT,
    encounterId STRING,
    month INT,
    year INT,
    status STRING
  );
''')

medication_administration_table = client.get_table(f"{dataset.dataset_id}.medication_administration")
medication_administration_table_name = f"{dataset.dataset_id}.{medication_administration_table.table_id}"
print(f"Successfully created table {medication_administration_table}")

Successfully created table assignment-1-448801.portfolio_data_modeling.medication_administration


We will just be joining dimension tables to the ids, so we don't have to add a search index here.

In [ ]:
# Populate the table by extracting the surrogate ids from the dimension tables
client.query(
f'''
  INSERT INTO {medication_administration_table_name}
  (medicationAdministrationId, patientId, medicationId, encounterId, month, year, status)
  (
    SELECT DISTINCT
      medication_request.id as medicationAdministrationId,
      patient.id,
      medication.medicationId,
      encounter.encounterId as encounterId,
      -- Extract month and year for analysis
      EXTRACT(MONTH FROM CAST(medication_request.authoredOn AS TIMESTAMP)) as month,
      EXTRACT(YEAR FROM CAST(medication_request.authoredOn AS TIMESTAMP)) as year,
      medication_request.status
    FROM `bigquery-public-data.fhir_synthea.medication_request` as medication_request
      JOIN `portfolio_data_modeling.encounter` as encounter
      ON medication_request.context.encounterId = encounter.encounterId
      -- Match patientId to retrieve id
      JOIN `portfolio_data_modeling.patient` as patient
      ON patient.patientId = subject.patientId
      -- Match medicationId to retrieve id
      JOIN `portfolio_data_modeling.medication` as medication
      ON medication.medicationCode = medication_request.medication.codeableConcept.`coding`[SAFE_OFFSET(0)].code
  );
''').result()

## Data Model Validation
To ensure we have created our fact and dimension tables correctly and accurately, we need to validate. We need to test to:

- Ensure linkages between tables are present and valid
  - Google BigQuery does not support foreign keys so this functionality is not included or tested
- Ensure expected values in the fact tables are not missing from the dimension tables

In [ ]:
client.query(f'''
  SELECT *
  FROM {medication_administration_table_name} me
    JOIN {encounter_table_name} e
    ON me.encounterId = e.encounterId
      JOIN {encounter_type_table_name} et
      ON e.encounterTypeId = et.encounterTypeId
    JOIN {patient_table_name} p
    ON me.patientId = p.id
    JOIN {medication_table_name} m
    ON m.medicationId = me.medicationId
  LIMIT 10
''').to_dataframe()

,medicationAdministrationId,patientId,medicationId,encounterId,month,year,status,encounterId_1,encounterTypeId,patientId_1,...,encounterTypeId_1,encounterTypeCode,encounterTypeName,id,patientId_2,firstName,lastName,medicationId_1,medicationCode,medicationName
0,3ed8a306-3f10-4f7d-a07b-d2e43237432a,931989,136,bf2b0567-b977-4d6c-9ebd-6bd0c5c3006d,1,1914,active,bf2b0567-b977-4d6c-9ebd-6bd0c5c3006d,11,931989,...,11,185345009,Encounter for symptom,931989,cab86271-589f-45ab-9537-fac69e4609b5,Leif534,Koelpin146,136,895994,120 ACTUAT Fluticasone propionate 0.044 MG/ACT...
1,3ed8a306-3f10-4f7d-a07b-d2e43237432a,931989,136,bf2b0567-b977-4d6c-9ebd-6bd0c5c3006d,1,1914,active,bf2b0567-b977-4d6c-9ebd-6bd0c5c3006d,12,931989,...,12,185345009,Encounter for symptom (procedure),931989,cab86271-589f-45ab-9537-fac69e4609b5,Leif534,Koelpin146,136,895994,120 ACTUAT Fluticasone propionate 0.044 MG/ACT...
2,5492c0cf-e3a6-40a1-8f49-fe76733bad6e,1096412,99,92569122-1770-42ad-a736-7a05dbc06925,10,1916,stopped,92569122-1770-42ad-a736-7a05dbc06925,38,1096412,...,38,50849002,Emergency Room Admission,1096412,ee994499-17c6-40c3-a112-fb2ef3d0054f,Herbert830,Senger904,99,563026,Diazepam [Valium]
3,5492c0cf-e3a6-40a1-8f49-fe76733bad6e,1096412,99,92569122-1770-42ad-a736-7a05dbc06925,10,1916,stopped,92569122-1770-42ad-a736-7a05dbc06925,37,1096412,...,37,50849002,Emergency Encounter,1096412,ee994499-17c6-40c3-a112-fb2ef3d0054f,Herbert830,Senger904,99,563026,Diazepam [Valium]
4,5492c0cf-e3a6-40a1-8f49-fe76733bad6e,1096412,99,92569122-1770-42ad-a736-7a05dbc06925,10,1916,stopped,92569122-1770-42ad-a736-7a05dbc06925,39,1096412,...,39,50849002,Emergency room admission (procedure),1096412,ee994499-17c6-40c3-a112-fb2ef3d0054f,Herbert830,Senger904,99,563026,Diazepam [Valium]
5,fe36fcd0-a0bd-416f-89c1-a86396c5e131,91838,108,3e37c304-87fc-44b8-b57b-ff676d4c2d83,10,1916,active,3e37c304-87fc-44b8-b57b-ff676d4c2d83,11,91838,...,11,185345009,Encounter for symptom,91838,13ed6584-146f-4216-82fa-b3b8310c63e8,Kelley882,Russel238,108,745679,200 ACTUAT Albuterol 0.09 MG/ACTUAT Metered Do...
6,fe36fcd0-a0bd-416f-89c1-a86396c5e131,91838,108,3e37c304-87fc-44b8-b57b-ff676d4c2d83,10,1916,active,3e37c304-87fc-44b8-b57b-ff676d4c2d83,12,91838,...,12,185345009,Encounter for symptom (procedure),91838,13ed6584-146f-4216-82fa-b3b8310c63e8,Kelley882,Russel238,108,745679,200 ACTUAT Albuterol 0.09 MG/ACTUAT Metered Do...
7,5236887f-be87-4d2d-b3c5-4c2d98db0f13,592068,108,aa1979fb-5209-4e68-ae1a-087b96beb6a9,11,1920,active,aa1979fb-5209-4e68-ae1a-087b96beb6a9,12,592068,...,12,185345009,Encounter for symptom (procedure),592068,80d70a5d-6af6-4b40-9cac-71ceca373263,Jacque955,Kulas532,108,745679,200 ACTUAT Albuterol 0.09 MG/ACTUAT Metered Do...
8,5236887f-be87-4d2d-b3c5-4c2d98db0f13,592068,108,aa1979fb-5209-4e68-ae1a-087b96beb6a9,11,1920,active,aa1979fb-5209-4e68-ae1a-087b96beb6a9,11,592068,...,11,185345009,Encounter for symptom,592068,80d70a5d-6af6-4b40-9cac-71ceca373263,Jacque955,Kulas532,108,745679,200 ACTUAT Albuterol 0.09 MG/ACTUAT Metered Do...
9,07e6870e-68b6-43d9-b0f9-b38fb7693da5,1023662,99,104b1e7e-47ef-445e-ab45-e178521dce83,5,1924,stopped,104b1e7e-47ef-445e-ab45-e178521dce83,16,1023662,...,16,185349003,Encounter for 'check-up',1023662,dec92f5b-5524-4e2a-ad60-3e5212ea3351,Gary33,Kreiger457,99,563026,Diazepam [Valium]


In [ ]:
client.query(f'''
  -- Count duplicates
  SELECT medicationAdministrationId, COUNT(*)
  FROM `portfolio_data_modeling.medication_administration`
  -- Group duplicates together
  GROUP BY medicationAdministrationId
  -- Limit to duplicated rows
  HAVING COUNT(*) > 1;
''').to_dataframe()

,medicationAdministrationId,f0_


## The Analysis
Now we are finally ready to answer the question, "how many prescriptions were for metformin each month?"

In [ ]:
metformin_prescriptions_per_month = client.query(f'''
  -- Count the total prescriptions
  SELECT COUNT(*) as totalPrescriptions,
    -- Get month and year for aggregations
    medication_administration.month, medication_administration.year
  FROM `portfolio_data_modeling.medication_administration` medication_administration
    -- Join with the medication dimension table to look up the medication name
    JOIN `portfolio_data_modeling.medication` medication
    ON medication_administration.medicationId = medication.medicationId
  -- Filter to only get metformin prescriptions
  WHERE SEARCH(LOWER(medication.medicationName), 'metformin')
  -- Eliminate any statuses that imply the prescription did not go through
  AND status not in ('cancelled', 'entered-in-error', 'draft', 'unknown', 'on-hold')
  -- Group by the year and month to get counts of prescriptions per month
  GROUP BY medication_administration.year, medication_administration.month
  ORDER BY medication_administration.year, medication_administration.month
''').to_dataframe()

metformin_prescriptions_per_month

,totalPrescriptions,month,year
0,1,12,1928
1,1,1,1929
2,1,1,1931
3,2,2,1931
4,2,4,1931
...,...,...,...
1034,90,5,2019
1035,93,6,2019
1036,102,7,2019
1037,104,8,2019


### Visualization
For the x-axis we need to combine the month and year as a unique id for each row.

To visualize this data, I have chosen a simple bar graph. This makes it easy to read and see the difference in metformin prescriptions month to month and see the trends over time.

In [ ]:
import plotly.express as px

# combine month and year for indexing
metformin_prescriptions_per_month['year_month'] = metformin_prescriptions_per_month['year'].astype(str) + '-' + metformin_prescriptions_per_month['month'].astype(str).str.zfill(2)

fig = px.bar(metformin_prescriptions_per_month, x='year_month', y="totalPrescriptions", title = 'Metformin Prescriptions Per Month', height=600, text_auto=True)
fig.update_layout(xaxis=dict(title='Year, Month'), yaxis=dict(title='Total Prescriptions'))
fig.show()

# Resources
HL7. 2023. "Welcome to FHIR." Last Modified March 26, 2023. http://hl7.org/fhir/.

Mermaid. n.d. "Mermaid Diagramming and charting tool." Accessed February 3, 2025. https://mermaid.js.org/.

Kimball, Ralph, and Ross, Margy. 2013. "Data Warehousing, Business Intelligence, and Dimensional Modeling Primer." In *The Data Warehouse Toolkit : The Definitive Guide to Dimensional Modeling,* 1-18. Newark: John Wiley & Sons, Incorporated. ProQuest Ebook Central.

Google Cloud. 2025. "Introduction to Search in BigQuery." BigQuery Documentation. Last modified February 6, 2025. https://cloud.google.com/bigquery/docs/search-intro.